### 말뭉치 INTENT 훈련 및 사용자 사전 생성 작업
* 자료 로드
  1. 처음배우는 딥러닝 챗봇 data
  2. 등산로

In [2]:
import pandas as pd
import json
import folium
print(f"folium Version: {folium.__version__}")

folium Version: 0.14.0


#### 자료 로드
1. 등산로 자료

In [3]:

#geo_path = '../DABA/seoul-dong.geojson'
geo_path = '../../DATA/FRT000801/moutain_load.geojson'
try: 
    df_geo = json.load(open(geo_path, encoding='utf-8'))
except:
    df_geo = json.load(open(geo_path, econding='utf-8-sig'))   # return dict

df_geo.keys()
df_geo
#df_geo['features']

import re
from re import search, findall, match, sub

# 등산로 명 뽑기
#print(df_geo.keys())
#print(type(df_geo))
print("df #: ", len(df_geo['features'][:]))
var_mountains = [ df_geo['features'][i]['properties']['MNTN_NM'] for i in range(len(df_geo['features'][:])) ]
var_loads = [ df_geo['features'][i]['properties']['PMNTN_NM'] for i in range(len(df_geo['features'][:])) ]

#print(mountains[0:5])

set_mountains = list(filter(None, set(var_mountains)))
set_loads = list(filter(None, set(var_loads)))
set_loads = [ i.replace(",", "-") for i in set_loads ]
print("mountain #: ", len(set_mountains))
print("loads #: ", len(set_loads))


# 산, 등산로 너무 많음, 랜덤추출로 100개로 줄이기
import random

random.seed(1)

extract_num = 100
rand_mtn = random.sample(range(1,len(set_mountains)),extract_num)
rand_load = random.sample(range(1,len(set_loads)),extract_num)
#rand_mtn = random.sample(range(0,len(set_mountains)),len(set_mountains))
#rand_load = random.sample(range(0,len(set_loads)),len(set_loads))

print(len(rand_mtn))
print(len(rand_load))

#print()

df #:  57890
mountain #:  2210
loads #:  4879
100
100


### mtn_corpus.txt 만들기

In [ ]:
# with open("../../DATA/corpus.txt", 'r', encoding='utf8') as f:
#     ori_lines = f.readlines()

#### mtn_train_data.csv 만들기

In [4]:
######### mtn_train_data.csv 만들기

# 1. 0,1,4 라벨만 추출
ex_14 = []
with open("../../DATA/total_train_data1.csv", 'r', encoding='utf8') as f:
    ori_lines = f.readlines()
    #print(ori_lines[0:5])
    for i, line in enumerate(ori_lines):
        query = line.split(",")[0]
        label = line.split(",")[1]
        #print(label.strip())
        if (label.strip() != "2") and (label.strip() != "3"):
            ex_14.append(line)
            #print(line)
        #if i == 10: break

#print(ex_14[0:5])


# 2. 추출값 먼저 write
with open("../../DATA/mtn_train_data.csv", 'w', encoding='utf8') as f:
   f.writelines(ex_14)

# 2.1 조회:2, 등산로 명 데이터 추가            
with open("../../DATA/mtn_train_data.csv", 'a', encoding='utf8') as f:

   # 일단 산개수만큼
   for i in range(len(set_mountains)):
           a = re.search('[-._()~,0-9a-zA-Z]', set_mountains[i])
           b = re.search('[-._()~,0-9a-zA-Z]', set_loads[i])
           # 특수문자 포함 단어 제외
           if (a == None) and (b == None) and (len(set_mountains[i].split()) == 1) and (len(set_loads[i].split()) == 1 ):
               f.write(f'{set_mountains[i]} {set_loads[i]}, 2')
               f.write('\n')
   # 나머지 로드개수만큼
   for i in range(len(set_loads) - len(set_mountains)-1,len(set_loads),1):
           a = re.search('[-._()~,0-9a-zA-Z]', '관악산')
           b = re.search('[-._()~,0-9a-zA-Z]', set_loads[i])
           # 특수문자 포함 단어 제외
           if (b == None) and (len(set_loads[i].split()) == 1 ):
               f.write(f'관악산 {set_loads[i]}, 2')
               f.write('\n')


# 2.2 조회:2 시간
with open("../../DATA/mtn_train_data.csv", 'a', encoding='utf8') as f:
    for i in range(24):
        for j in range(60):
         f.write(f'{i}시 {j}분, 2')
         if i !=49: f.write('\n')



# 2.3 선택:3, 등산로 리스트중 선택 기능을 위한 자료 추가
with open("../../DATA/mtn_train_data.csv", 'a', encoding='utf8') as f:
    for i in range(50):
         f.write(f'{i}번, 3')
         f.write('\n')
with open("../../DATA/mtn_train_data.csv", 'a', encoding='utf8') as f:
    for i in range(50):
         f.write(f'{i}, 3')
         if i !=49: f.write('\n')



#### mtn_user_dict.tsv 만들기

In [7]:
######### mtn_user_dict.tsv 만들기

# 1. 기존 정보 불러오기
with open("../../DATA/user_dic1.tsv", 'r', encoding='utf8') as f:
    ori_lines = f.readlines()

with open("../../DATA/mtn_user_dict.tsv", 'w', encoding='utf8') as f:
    f.writelines(ori_lines)
    f.write('\n')
    # 산 출력
    for word in set_mountains:
        b = re.search('[-._()~,0-9a-zA-Z]', word)
        if b == None:
            if len(word.split()) > 1:
                for wd in word.split():
                    # 여러개면 분리
                    print_word = wd + "\t" + "NNP"
                    f.write(print_word)
                    f.write('\n')
            else:
                print_word = word + "\t" + "NNP"
                f.write(print_word)
                f.write('\n')

    # 길 이름 출력
    for i, sent in enumerate(set_loads):
        a = re.search('[-._()~,0-9a-zA-Z]', sent)
        # 특수문자 포함 단어 제외
        if a == None:
            # 여러개면 분리
            if len(sent.split()) > 1:
                for ww in sent.split():
                    print_word = str(ww) + "\t" + "NNP"
                    f.write(print_word)
                    f.write('\n')
                    #if i != len(set_loads)-1: f.write('\n')
            else:
                print_word = str(sent) + "\t" + "NNP"
                f.write(print_word)
                f.write('\n')
    # 길이름 분리O
    # for i, sent in enumerate(set_loads):
    #     a = re.sub('[-.()~,]', ' ', sent)
    #     for j in a.split():
    #         print_word = str(j) + "\t" + "NNP"
    #         f.write(print_word)
    #         if i != len(set_loads)-1: f.write('\n')

### mtn_ner_train.txt 만들기

In [8]:
# ; 수인산 성신리구간 찾아줘
# $<수인산:LC> <성산리구간:LC> 찾아줘
# 1 수인산  NNP B_LC
# 2 성신리구간  NNP B_LC
#12	찾	VV	O
#12	아	EC	O
#12	주	VX	O
#12	어	EC	O

import re
import numpy as np

#def print_ner_lines(f, mtn, load, k, n):
def print_ner_lines( f, mtn, load):

    #print(load)
    load_split = re.split('[,/ -]', load)

    a = re.search('[-._()~,0-9a-zA-Z]', mtn)
    b = re.search('[-._()~,0-9a-zA-Z]', load)

    # 특수문자 포함 단어 제외
    if (a == None) and (b == None) and (len(load.split()) == 1):

        f.write(f'; {mtn} {load}\n')
        f.write(f'$ <{mtn}:LC> <{load}:LC>\n')
        f.write(f'1\t{mtn}\tNNP\tB_LC\n')
        f.write(f'2\t{load_split[0]}\tNNP\tB_LC\n')
        f.write('\n')
        # f.write(f'3\t찾\tVV\tO\n')
        # f.write(f'4\t아\tEC\tO\n')
        # f.write(f'5\t주\tVX\tO\n')
        # f.write(f'6\t어\tEC\tO\n')
    else:
        pass
        #f.write('\n')

#print_ner_lines("수인산", "성신리구간")

# 학습 파일 불러오기
with open('../../DATA/ner_train1.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()


with open('../../DATA/mtn_ner_train_fullversion.txt', 'w', encoding='utf-8') as f:
    f.writelines(lines)
    f.write('\n\n')

    # # small 100mtn, 100loads
    # for k, i in enumerate(rand_mtn):
    #     for j in rand_load:            
    #         print_ner_lines(f,set_mountains[i],set_loads[j])
    #     #if k == 3: break
    
    # full mtn, loads 2000 * 4000
    for i in set_mountains:
        run_loads = np.random.choice(set_loads, 2, replace=False)
        for j in run_loads:            
            print_ner_lines(f,i,j)
        
    
# for i in rand_mtn:
#     for j in rand_load:
#         print_ner_lines(set_mountains[i],set_loads[j])


In [5]:
import numpy as np
sample1 = np.random.choice(['A', 'B', 'C', 'D', 'E'], 3, replace=False)
sample2 = np.random.choice(['A', 'B', 'C', 'D', 'E'], 3, replace=False)

print(sample1)
print(sample2)

['C' 'E' 'B']
['E' 'A' 'C']


In [3]:
import re

text = "a-b,c d"
a = re.split('[,/ -]', text)
print(a)


['a', 'b', 'c', 'd']


In [2]:
a=['abc']
print(a[0])

abc


In [ ]:
a = "창동"
a.split()

In [ ]:
list1 = ["창동 초안산-창동 정화도서관구간",
        "뉴.정일그린타운뒤구간",
        "대승사입구구간",
        "득량면삼정리(오도치재)-미력면초당",
        "철도공원앞~김천일장군묘구간",
        "철도공원앞,김천일장군묘구간"]

for i in list1:
    a = re.sub('[-.()~,]', ' ', i)
    b = a.split()
    print("="*30)
    print(a)
    print(b)
    for j in b:
        print(j)


In [6]:
for i in list1:
    a = re.search('[-.()~,]', i)
    print("="*30)
    if a == None:
       print(a)


None


In [9]:
len1 = 10
len2 = 21

for i in range(len1):
    print(i)

for i in range(len2-len1-1,len2,1):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
